# Lab Fees

In [1]:
#read in the data and the file with all the duplicates
import pandas as pd
import numpy as np

lab_dups = pd.read_csv('lab_dups.csv',error_bad_lines=False)
lab_newcols = lab_dups['newcols']
del lab_dups['newcols']

lab = pd.read_csv('lab - not cleaned.csv', encoding = "ISO-8859-1", sep = ";")
lab = lab.replace("N/A", np.nan)

b'Skipping line 59: expected 12 fields, saw 13\n'


In [2]:
def comb_dups(dups, dat, newcols):
    for i in range(0, dups.shape[0]):
        newcol_items = []
    
        colslist = dups.iloc[i].unique() 
        colslist = [x for x in colslist if str(x) != 'nan'] 
        for row in range(0, dat.shape[0]):
            vals = dat[colslist].iloc[row].unique() 
            vals = [x for x in vals if str(x) != 'nan']
            if len(vals) == 0: 
                newcol_items.append(np.nan) 
            else: 
                vals = [float(v) for v in vals] 
                newcol_items.append(np.nanmax(vals))

        dat = dat.drop(colslist, axis = 1) 
        dat[newcols[i]] = newcol_items 
    return(dat)
    
lab = comb_dups(lab_dups, lab, lab_newcols)

In [3]:
lab.to_csv('lab.csv',index = False)

### Data Analysis

In [4]:
from scipy.stats import ttest_ind

def summarize(dat):
    meds = []
    mean = []
    sd = []
    median = []
    mini = []
    maxi = []
    rangeo = []
    n = []
    no_urban = []
    mean_urban = []
    mean_rural = []
    no_rural = []
    ttests = []
    high_outliers = []
    low_outliers = []

    for col in [c for c in dat.columns.values if c not in ['hosp_name','urban/rural']]:
    
        vals = [float(i) for i in dat[col] if str(i) != 'nan']
    
        if len(vals) >= 10:
            #summary statistics
            meds.append(col)
            mean.append(np.mean(vals))
            sd.append(np.std(vals))
            median.append(np.median(vals))
            mini.append(np.nanmin(vals))
            maxi.append(np.nanmax(vals))
            rangeo.append(np.nanmax(vals) - np.nanmin(vals))
            n.append(len(vals))
            
            #ttest
            sub = dat[dat[col].notnull()]
            ttest = ttest_ind(sub[col].loc[sub['urban/rural'] == 'U'], sub[col].loc[sub['urban/rural'] == 'R'])[1]
            no_urban.append(len(sub[col].loc[sub['urban/rural'] == 'U']))
            mean_urban.append(np.mean(sub[col].loc[sub['urban/rural'] == 'U']))
            no_rural.append(len(sub[col].loc[sub['urban/rural'] == 'R']))
            mean_rural.append(np.mean(sub[col].loc[sub['urban/rural'] == 'R']))
            ttests.append(ttest)
            
            #identifying outliers
            high_outlier = []
            low_outlier = []
    
            for hosp in dat['hosp_name']:
                if str(dat[col].loc[dat.hosp_name == hosp].values[0]) != 'nan':
                    val = dat[col].loc[dat.hosp_name == hosp].values[0]
                    
                    mad = 1.4826 * np.median(abs(vals - np.median(vals)))

                    if np.less(np.median(vals) + 3 * mad, val):
                        outlier = [hosp, ": ",str(val)]
                        high_outlier.append("".join(outlier))
            
                    if np.less(val, np.median(vals) - 3 * mad):
                        outlier = [hosp, ": ",str(val)]
                        low_outlier.append("".join(outlier))
    
            high_outliers.append(high_outlier)
            low_outliers.append(low_outlier)

    summary = pd.DataFrame({'meds': meds,
                       'mean': mean,
                       'sd': sd,
                       'median': median,
                       'min': mini,
                       'max': maxi,
                       'range': rangeo,
                       'no urban': no_urban,
                       'mean urban': mean_urban,
                       'no rural': no_rural,
                       'mean rural': mean_rural,
                       't-test': ttests,
                       'N': n,
                       'high outliers': high_outliers,
                       'low outliers': low_outliers})
    return(summary)

summary1 = summarize(lab)

In [5]:
summary1.to_csv('summary1.csv')

# Diagnostics Fees

In [6]:
import pandas as pd
import numpy as np

diag_dups = pd.read_csv('diag_dups.csv',error_bad_lines=False, encoding = "ISO-8859-1", sep = ";")
diag_newcols = diag_dups['newcols']
del diag_dups['newcols']

diag = pd.read_csv('diag - not cleaned.csv', encoding = "ISO-8859-1", sep = ";")
diag = diag.replace("N/A", np.nan)
diag = diag.replace("N/A ", np.nan)
diag = diag.replace("", np.nan)

#removing duplicates
diag = comb_dups(diag_dups, diag, diag_newcols)

In [7]:
diag.to_csv('diag.csv')

In [8]:
#summarizing
summary2 = summarize(diag)

C:\Users\asus\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3126: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\asus\Anaconda3\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\asus\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\asus\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\asus\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [9]:
summary2.to_csv('summary2.csv')

## Summary of Outliers

In [10]:
summary = pd.DataFrame({
    'vars': summary1['meds'],
    'high outliers': summary1['high outliers'],
    'low outliers': summary1['low outliers']
})

summary = summary.append(pd.DataFrame({
    'vars': summary2['meds'],
    'high outliers': summary2['high outliers'],
    'low outliers': summary2['low outliers']
}))

In [11]:
lab_code = pd.read_csv('lab_code.csv')
diag_code = pd.read_csv('diag_code.csv')

summary['test'] = np.nan

for row in range(0, summary.shape[0] - 1):
    if summary['vars'].iloc[row] in list(lab_code['Variable']):
        summary['test'].iloc[row] = "".join(lab_code['Label Name'].loc[lab_code.Variable == summary['vars'].iloc[row]])
    else:
        summary['test'].iloc[row] = "".join(diag_code['Label Name'].loc[diag_code.Variable == summary['vars'].iloc[row]])

C:\Users\asus\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
summary.head()

,high outliers,low outliers,vars,test
0,[Don Emilio Del Valle Memorial Hospital: 559.0],[],abg,Arterial Blood Gas (ABG)
1,[],[],bil_indirect,Bilirubin - Indirect
2,"[Bataan General Hospital: 400.0, Culion Sanita...",[],glc.exam.OGT.urine,Blood Glucose Exam - Oral Glucose Tolerance Te...
3,[],[],glc.exam.OGT.100,Blood Glucose Exam - Oral Glucose Tolerance Te...
4,[Rizal Medical Center: 200.0],[],comp03,Complement 3 (C3)


In [13]:
hosps = list(lab['hosp_name'])

high_outliers_lists = []
low_outliers_lists = []

for hosp in hosps:
    highs = []
    lows = []
    
    for row in range(0, summary.shape[0] - 1):
        high_outs = '\t'.join(summary['high outliers'].iloc[row])
        low_outs = '\t'.join(summary['low outliers'].iloc[row])
        
        var = summary['vars'].iloc[row]
        if hosp in high_outs:
            if var in list(lab.columns.values):
                fee = [summary['test'].iloc[row], ": ", list(lab[var].loc[lab.hosp_name == hosp])[0]]
            else:
                fee = [summary['test'].iloc[row], ": ", list(diag[var].loc[diag.hosp_name == hosp])[0]]
            highs.append("".join(str(e) for e in fee))
        
        if hosp in low_outs:
            if var in list(lab.columns.values):
                fee = [summary['test'].iloc[row], ": ", list(lab[var].loc[lab.hosp_name == hosp])[0]]
            else:
                fee = [summary['test'].iloc[row], ": ", list(diag[var].loc[diag.hosp_name == hosp])[0]]
            lows.append("".join(str(e) for e in fee))
    
    high_outliers_lists.append(highs)
    low_outliers_lists.append(lows)
    
outlier_summary = pd.DataFrame({
    'hospitals': hosps,
    'high outliers': high_outliers_lists,
    'low outliers': low_outliers_lists
})

In [14]:
outlier_summary.to_csv('outlier summary.csv')